<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FP_MULTIPLE_LLM_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai -q
!pip install anthropic -q
!pip install colab-env -q

In [1]:
import os
from typing import Dict, Any, List, Optional
import google.generativeai as genai
from anthropic import Anthropic
import colab_env # For Google Colab

# Import necessary libraries
try:
    import google.generativeai as genai
    from anthropic import Anthropic
    from anthropic.types import MessageParam
    # Conditional import for Colab's userdata
    try:
        from google.colab import userdata
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False
        print("Running outside Google Colab. 'userdata.get' will not be available.")

except ImportError:
    print("\n[CRITICAL ERROR] Please install necessary libraries:")
    print("pip install google-generativeai anthropic")
    print("Exiting as API functionality cannot be demonstrated without them.")
    exit() # Exit if core libraries are missing


# --- 1. Core Components (Interfaces and Implementations with API Management) ---

class BaseModel:
    """Base class for Language Models and Reasoning Models."""
    def __init__(self, name: str, identifier: str, api_client: Any):
        self.name = name
        self.identifier = identifier # The specific model string (e.g., "models/gemini-2.0-flash")
        self.api_client = api_client # The actual API client instance (e.g., genai.GenerativeModel or anthropic.Anthropic)

    def generate(self, prompt: str, **kwargs) -> str:
        raise NotImplementedError("Subclasses must implement 'generate'")

class LargeLanguageModel(BaseModel):
    """Integrates with models/gemini-2.0-flash API."""
    def __init__(self, name: str, identifier: str, api_client: genai.GenerativeModel):
        super().__init__(name, identifier, api_client)
        if not isinstance(self.api_client, genai.GenerativeModel):
            print(f"Warning: Expected genai.GenerativeModel for {identifier}, but received {type(api_client)}. Mocking API calls.")
            self.api_client = None # Set to None to trigger mock

    def generate(self, prompt: str, temperature: float = 0.7, max_tokens: int = 500) -> str:
        """
        Makes an actual API call to Google Gemini.
        """
        print(f"[{self.name} ({self.identifier})] Attempting to generate response for prompt: '{prompt[:50]}...'")

        if self.api_client:
            try:
                generation_config = {
                    "temperature": temperature,
                    "max_output_tokens": max_tokens,
                }
                response = self.api_client.generate_content(
                    prompt,
                    generation_config=generation_config
                )
                if response and response.candidates:
                    return response.text
                elif response and response.prompt_feedback:
                    print(f"Prompt feedback from Gemini: {response.prompt_feedback}")
                    return f"API did not generate content due to safety concerns or other issues."
                return f"API generated an empty response."
            except Exception as e:
                print(f"Error calling Gemini API for {self.identifier}: {e}")
                return self._mock_generate(prompt)
        else:
            return self._mock_generate(prompt) # Fallback to mock if client not initialized

    def _mock_generate(self, prompt: str) -> str:
        """Mock response when API is not available or fails."""
        print(f"[{self.name} ({self.identifier})] (MOCK) Generating response for prompt: '{prompt[:50]}...'")
        if "flight planning" in prompt.lower() and "origin" in prompt.lower():
            return f"[{self.name}] (MOCK) User is asking about flight planning. Need to retrieve flight data for: {prompt}. Consider weather and routes."
        elif "explain" in prompt.lower():
            return f"[{self.name}] (MOCK) Explanation: This is a complex topic. Let me break it down based on '{prompt}'."
        return f"[{self.name}] (MOCK) Generated a response based on: '{prompt}'"


class LargeReasoningModel(BaseModel):
    """Integrates with claude-opus-4-20250514 API."""
    def __init__(self, name: str, identifier: str, api_client: Anthropic):
        super().__init__(name, identifier, api_client)
        if not isinstance(self.api_client, Anthropic):
            print(f"Warning: Expected anthropic.Anthropic for {identifier}, but received {type(api_client)}. Mocking API calls.")
            self.api_client = None # Set to None to trigger mock

    def generate(self, prompt: str, complexity_level: str = "high") -> str:
        """
        Makes an actual API call to Anthropic Claude.
        """
        print(f"[{self.name} ({self.identifier})] Attempting to perform reasoning for prompt: '{prompt[:50]}...' (Complexity: {complexity_level})")

        if self.api_client:
            try:
                messages: List[MessageParam] = [
                    {"role": "user", "content": prompt}
                ]
                response = self.api_client.messages.create(
                    model=self.identifier,
                    max_tokens=1000,
                    messages=messages,
                    temperature=0.1,
                )
                if response and response.content:
                    return "".join(block.text for block in response.content if hasattr(block, 'text'))
                return f"API generated an empty response."
            except Exception as e:
                print(f"Error calling Anthropic API for {self.identifier}: {e}")
                return self._mock_generate(prompt)
        else:
            return self._mock_generate(prompt) # Fallback to mock if client not initialized

    def _mock_generate(self, prompt: str) -> str:
        """Mock response when API is not available or fails."""
        print(f"[{self.name} ({self.identifier})] (MOCK) Performing reasoning for prompt: '{prompt[:50]}...'")
        if "weather for flight" in prompt.lower():
            return f"[{self.name}] (MOCK) LRM reasoned: Checked weather patterns. Identified potential turbulence near destination. Recommend alternative altitude and departure time."
        elif "optimize route" in prompt.lower():
            return f"[{self.name}] (MOCK) LRM reasoned: Analyzed fuel consumption, air traffic data, and historical delays. Optimized route to minimize cost and time, suggesting waypoint adjustments."
        return f"[{self.name}] (MOCK) LRM reasoned extensively on: '{prompt}'"

# (Memory, VectorDatabase, SemanticDatabase, SearchEngine, RetrievalAgent classes remain unchanged)
class Memory:
    """Manages short-term and long-term memory."""
    def __init__(self):
        self.short_term_memory: List[str] = []
        self.long_term_memory: List[str] = [] # Could be a more complex data structure

    def add_short_term(self, entry: str):
        self.short_term_memory.append(entry)
        # Keep short-term memory limited, e.g., last N interactions
        if len(self.short_term_memory) > 10:
            self.short_term_memory.pop(0)

    def add_long_term(self, entry: str):
        self.long_term_memory.append(entry)
        # In a real system, this might involve embeddings and indexing

    def retrieve_short_term(self, query: str = "") -> List[str]:
        # Simple retrieval; in real system, would be more sophisticated (e.g., semantic search)
        return [m for m in self.short_term_memory if query.lower() in m.lower()]

    def retrieve_long_term(self, query: str = "") -> List[str]:
        return [m for m in self.long_term_memory if query.lower() in m.lower()]

class VectorDatabase:
    """Mock Vector Database."""
    def query(self, embedding: List[float], top_k: int = 5) -> List[str]:
        print(f"Querying Vector DB with embedding...")
        # Simulate results related to flight data
        return ["flight_route_data_xyz_vdb", "aircraft_specs_abc_vdb"]

class SemanticDatabase:
    """Mock Semantic Database."""
    def query(self, natural_language_query: str) -> List[str]:
        print(f"Querying Semantic DB for: '{natural_language_query}'")
        # Simulate results related to aviation regulations or air traffic rules
        if "regulations" in natural_language_query:
            return ["Regulation 123.45: Airspace Class B rules for drones.", "Pilot licensing requirements document for commercial flights."]
        return ["semantic_info_1_sdb", "semantic_info_2_sdb"]

class SearchEngine:
    """Mock Search Engine (e.g., Google, X, LinkedIn)."""
    def search(self, query: str) -> List[str]:
        print(f"Searching web for: '{query}'")
        if "weather" in query and "paris" in query:
            return ["Current weather in Paris: Light rain, 15C, wind 10km/h.", "Flight delays due to adverse weather in Paris in May 2025."]
        return [f"Web result for '{query}' - Link A", f"Web result for '{query}' - Link B"]

# --- 2. Retrieval Agents ---

class RetrievalAgent:
    """Base class for retrieval agents."""
    def __init__(self, name: str):
        self.name = name

    def retrieve(self, query: str, **kwargs) -> List[str]:
        raise NotImplementedError("Subclasses must implement 'retrieve'")

class KnowledgeBaseRetrievalAgent(RetrievalAgent):
    """Retrieves from Vector and Semantic Databases."""
    def __init__(self, vector_db: VectorDatabase, semantic_db: SemanticDatabase):
        super().__init__("KnowledgeBaseAgent")
        self.vector_db = vector_db
        self.semantic_db = semantic_db

    def retrieve(self, query: str, query_type: str = "semantic") -> List[str]:
        results = []
        if query_type == "vector":
            # In a real system, 'query' would first be embedded
            mock_embedding = [0.1] * 768 # Placeholder embedding
            results.extend(self.vector_db.query(mock_embedding))
        elif query_type == "semantic":
            results.extend(self.semantic_db.query(query))
        return results

class SearchEngineRetrievalAgent(RetrievalAgent):
    """Retrieves from external search engines."""
    def __init__(self, search_engines: List[SearchEngine]):
        super().__init__("SearchEngineAgent")
        self.search_engines = search_engines

    def retrieve(self, query: str) -> List[str]:
        all_results = []
        for engine in self.search_engines:
            all_results.extend(engine.search(query))
        return all_results


# --- 3. Agentic Reasoning and Console ---

class AgenticReasoning:
    """Orchestrates LLM, LRM, and Retrieval Agents."""
    def __init__(self, llm_gemini_flash: LargeLanguageModel,
                 lrm_sonnet: LargeReasoningModel, memory: Memory,
                 kb_agent: KnowledgeBaseRetrievalAgent, se_agent: SearchEngineRetrievalAgent):
        self.llm_gemini_flash = llm_gemini_flash
        self.lrm_sonnet = lrm_sonnet
        self.memory = memory
        self.kb_agent = kb_agent
        self.se_agent = se_agent
        # We'll use Gemini Flash as the primary LLM
        self.current_llm = self.llm_gemini_flash

    def process_query(self, modified_input: str) -> str:
        """
        This method embodies the core Agentic Reasoning loop.
        It's a simplified representation of the complex internal logic.
        """
        print(f"\n[Agentic Reasoning] Processing input using {self.current_llm.identifier} and {self.lrm_sonnet.identifier}: '{modified_input}'")
        self.memory.add_short_term(f"User query: {modified_input}")

        # Step 1: Initial LLM interpretation and plan using models/gemini-2.0-flash
        llm_system_prompt_initial = "You are an AI assistant. Analyze the user query to determine information needs and initial reasoning steps. If the query is about flight planning, prioritize retrieving relevant data."
        initial_llm_thought = self.current_llm.generate(f"{llm_system_prompt_initial}\nUser query: {modified_input}")
        self.memory.add_short_term(f"LLM initial thought: {initial_llm_thought}")
        print(f"LLM Initial Thought: {initial_llm_thought}")

        # Step 2: Determine retrieval needs based on LLM's thought
        retrieval_query = ""
        if "flight data" in initial_llm_thought.lower() or "weather" in initial_llm_thought.lower() or "regulations" in initial_llm_thought.lower():
            retrieval_query = modified_input # Use original query or refine it
            print(f"Determined retrieval need for: '{retrieval_query}'")

            # Perform Retrieval
            kb_results = self.kb_agent.retrieve(retrieval_query, query_type="semantic") # Assume semantic for most queries
            se_results = self.se_agent.retrieve(retrieval_query)
            retrieved_info = "\n".join(kb_results + se_results)
            print(f"Retrieved Information: {retrieved_info[:200]}...")
            self.memory.add_short_term(f"Retrieved: {retrieved_info}")
        else:
            retrieved_info = "No specific retrieval needed."

        # Step 3: Engage LRM (claude-opus-4-20250514) if complex reasoning is required
        lrm_system_prompt = "You are a specialized reasoning engine (claude-opus-4-20250514). Analyze the gathered information and the user's original intent to formulate a precise answer. If the task is flight planning, provide a structured plan, considering all constraints and retrieved data."
        if "flight planning" in modified_input.lower() or "optimize" in initial_llm_thought.lower() or "analyze data" in initial_llm_thought.lower():
            reasoning_input = f"User query: {modified_input}\nInitial LLM thought: {initial_llm_thought}\nRetrieved info: {retrieved_info}"
            lrm_output = self.lrm_sonnet.generate(f"{lrm_system_prompt}\n{reasoning_input}")
            self.memory.add_short_term(f"LRM reasoning: {lrm_output}")
            print(f"LRM Output: {lrm_output}")
        else:
            lrm_output = "No specific LRM reasoning performed by claude-opus-4-20250514."

        # Step 4: Final LLM (models/gemini-2.0-flash) synthesis
        final_llm_prompt = f"Based on the user's original query: '{modified_input}', initial analysis: '{initial_llm_thought}', retrieved information: '{retrieved_info}', and specialized reasoning by {self.lrm_sonnet.identifier}: '{lrm_output}', synthesize a comprehensive and polite answer using {self.current_llm.name}."
        final_response = self.current_llm.generate(final_llm_prompt, max_tokens=1000)
        self.memory.add_long_term(f"Interaction: Query='{modified_input}', Response='{final_response}'")
        print(f"\n[Final Response] {final_response}")
        return final_response

class AgenticSearchConsole:
    """The top-level console orchestrating user interaction."""
    def __init__(self, gemini_client: genai.GenerativeModel, claude_client: Anthropic):
        # Initialize LLM with models/gemini-2.0-flash and the provided client
        self.llm_gemini_flash = LargeLanguageModel("Gemini 2.0 Flash", "gemini-2.0-flash", gemini_client) # Note: 'models/' prefix is often for API calls, 'gemini-2.0-flash' is common model name
        # Initialize LRM with claude-opus-4-20250514 and the provided client
        self.lrm_opus = LargeReasoningModel("Claude Opus 4", "claude-opus-4-20250514", claude_client)

        self.memory = Memory()
        self.vector_db = VectorDatabase()
        self.semantic_db = SemanticDatabase()
        self.search_engines = [SearchEngine(), SearchEngine()] # Mock multiple engines
        self.kb_agent = KnowledgeBaseRetrievalAgent(self.vector_db, self.semantic_db)
        self.se_agent = SearchEngineRetrievalAgent(self.search_engines)

        self.agentic_reasoning = AgenticReasoning(
            llm_gemini_flash=self.llm_gemini_flash,
            lrm_sonnet=self.lrm_opus, # Renamed to lrm_opus
            memory=self.memory,
            kb_agent=self.kb_agent,
            se_agent=self.se_agent
        )

    def run_query(self, user_query: str) -> str:
        print(f"\n--- [Agentic Search Console] Processing User Query: '{user_query}' ---")
        # Query Parsing (simple for now)
        parsed_query = user_query.strip()
        print(f"Query Parsed: '{parsed_query}'")

        # System Prompt for LLM
        system_prompt = "You are a helpful AI flight planning assistant. Your goal is to provide accurate and comprehensive flight plans and related information."
        modified_input = f"{system_prompt}\nUser: {parsed_query}"

        # Agentic Reasoning
        compiled_output = self.agentic_reasoning.process_query(modified_input)

        print(f"--- [Agentic Search Console] Query Complete ---")
        return compiled_output

# --- Example Usage (Adapted for Colab-like API key management) ---
if __name__ == "__main__":
    print("Setting up API clients...")

    # --- Gemini API Setup ---
    GOOGLE_API_KEY = None
    if IN_COLAB:
        try:
            GOOGLE_API_KEY = userdata.get('GEMINI')
            print("Successfully retrieved GEMINI_API_KEY from Colab environment variable.")
        except Exception as e:
            print(f"Could not retrieve GEMINI_API_KEY from Colab userdata: {e}. Checking environment variable.")
    if not GOOGLE_API_KEY:
        GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY") # Fallback to environment variable
        if GOOGLE_API_KEY:
            print("Successfully retrieved GOOGLE_API_KEY from environment variable.")
        else:
            print("WARNING: GOOGLE_API_KEY not found. Gemini API calls will be mocked.")

    gemini_client = None
    if GOOGLE_API_KEY:
        try:
            genai.configure(api_key=GOOGLE_API_KEY)
            gemini_client = genai.GenerativeModel('gemini-2.0-flash')
            print("Gemini GenerativeModel ('gemini-2.0-flash') client initialized.")
        except Exception as e:
            print(f"ERROR initializing Gemini client: {e}. Gemini API calls will be mocked.")
    else:
        print("No Google API Key found, Gemini client will not be initialized.")

    # --- Claude API Setup ---
    CLAUDE_API_KEY = None
    if IN_COLAB:
        try:
            CLAUDE_API_KEY = os.environ.get("CLAUDE3_API_KEY")
            print("Successfully retrieved CLAUDE3_API_KEY from Colab userdata.")
        except Exception as e:
            print(f"Could not retrieve CLAUDE3_API_KEY from Colab userdata: {e}. Checking environment variable.")
    if not CLAUDE_API_KEY:
        CLAUDE_API_KEY = os.environ.get("ANTHROPIC_API_KEY") # Fallback to environment variable
        if CLAUDE_API_KEY:
            print("Successfully retrieved ANTHROPIC_API_KEY from environment variable.")
        else:
            print("WARNING: ANTHROPIC_API_KEY not found. Claude API calls will be mocked.")

    claude_client = None
    if CLAUDE_API_KEY:
        try:
            claude_client = Anthropic(api_key=CLAUDE_API_KEY)
            print("Anthropic Claude client initialized.")
        except Exception as e:
            print(f"ERROR initializing Anthropic client: {e}. Claude API calls will be mocked.")
    else:
        print("No Anthropic API Key found, Claude client will not be initialized.")

    # Instantiate the console with the prepared API clients
    # This line creates the main console object that orchestrates everything.
    console = AgenticSearchConsole(gemini_client=gemini_client, claude_client=claude_client)
    print('\n')
    print("\nAgenticSearchConsole initialized. Running your specific query...")
    print('\n')

    # --- Running Your Specific Query: Montreal to Shanghai ---
    your_query = "Plan a real flight from Montreal, Canada to Shanghai, China for May 25th, 2025. Consider typical air traffic and historical weather patterns for that time of year, and please list the key waypoints along the route."

    print(f"\n--- Running Query: {your_query} ---")
    output = console.run_query(your_query)
    print(f"\n--- Output ---")
    print(output)
    print("--- End of Output ---")

    # --- Displaying Memory Contents after your query ---
    print("\n--- Current Short-Term Memory ---")
    print(console.agentic_reasoning.memory.retrieve_short_term())
    print('\n')

    print("\n--- Current Long-Term Memory ---")
    print(console.agentic_reasoning.memory.retrieve_long_term())
    print('\n')

Mounted at /content/gdrive
Setting up API clients...
Successfully retrieved GEMINI_API_KEY from Colab environment variable.
Gemini GenerativeModel ('gemini-2.0-flash') client initialized.
Successfully retrieved CLAUDE3_API_KEY from Colab userdata.
Anthropic Claude client initialized.



AgenticSearchConsole initialized. Running your specific query...



--- Running Query: Plan a real flight from Montreal, Canada to Shanghai, China for May 25th, 2025. Consider typical air traffic and historical weather patterns for that time of year, and please list the key waypoints along the route. ---

--- [Agentic Search Console] Processing User Query: 'Plan a real flight from Montreal, Canada to Shanghai, China for May 25th, 2025. Consider typical air traffic and historical weather patterns for that time of year, and please list the key waypoints along the route.' ---
Query Parsed: 'Plan a real flight from Montreal, Canada to Shanghai, China for May 25th, 2025. Consider typical air traffic and hist